<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_03_Arboles/03C_%7C_Introducci%C3%B3n_Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRO: Random Forest
Introducción a modelos de bagging de árboles de decisión (Random Forests).

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1jRg7Dk2y_2_fxnC_Jpj5aWcqgW9t1KAd7izdmWzv9Sk/edit?usp=sharing)
*   [Enlace a este notebook](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_03_Arboles/03C_%7C_Introducci%C3%B3n_Random_Forests.ipynb)




## 0101 Qué es un Random Forest?
Es una agregación de árboles de decisión de tipo bagging (**B**ootstrapping + Aggregation).

## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn import tree
from sklearn import ensemble
from sklearn.metrics import accuracy_score

## 0103 Carga el dataset del Titanic
Volvamos con el Dataset del Titanic

Vamos a comparar el rendimiento de un Random Forest con los modelos de árboles de decisión que creamos en el [notebook 3B](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_03_Arboles/03B_%7C_Repaso_II_sobre_%C3%81rboles_Decisi%C3%B3n.ipynb).

Este es el link al archivo raw: https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/titanic.csv.

Importa los datos en un dataframe llamado **dataframe df_titanic**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/titanic.csv'

# Importa csv
df_titanic = pd.read_csv(url)

# Visualización primeras filas
df_titanic.head()

## 0104 Limpieza del Dataset
Limpiamos los nulos y creamos la variable Title como en el [Notebook 3B](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_03_Arboles/03B_%7C_Repaso_II_sobre_%C3%81rboles_Decisi%C3%B3n.ipynb).

In [ ]:
# Creamos una nueva variable Título (es opcional, pero podría ayudar al modelo)
df_titanic['Title'] = [name.split(",")[1].split(".")[0][1:] for name in df_titanic['Name']]
df_titanic.head(2)

In [ ]:
# Columnas con las que nos quedamos
cols = ['Survived', 
        'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title']

# De ellas, cuáles son categóricas
cat_cols = ['Sex', 'Embarked', 'Title']

# Visualizamos las columnas con las que nos hemos quedado
df_titanic[cols].head()

In [ ]:
# Rellenamos los nulos de las variables numéricas (edad)
df_titanic[cols].isna().sum()

In [ ]:
# Rellenamos los nulos de la variable edad de una forma avanzada:
# calculando la media para cada título

filtro_edad_nula = df_titanic['Age'].isna()
filtro_edad_no_nula = df_titanic['Age'].notnull()

# Rellenamos los nulos en función del título 
for titulo in df_titanic['Title'].unique():
  filtro_titulo = df_titanic['Title'] == titulo

  if len(df_titanic[((filtro_titulo) & (filtro_edad_no_nula))]) > 2:
    print(titulo)
    df_titanic.loc[((filtro_titulo)&(filtro_edad_nula)), 'Age'] = df_titanic.loc[((filtro_titulo)&(filtro_edad_no_nula)), 'Age'].median()
  
# Rellenamos los que sigan siendo nulos (títulos solo presentes entre gente sin la edad informada)
df_titanic['Age'] = df_titanic['Age'].fillna(df_titanic['Age'].median())

In [ ]:
# Eliminamos los nulos de la variable Embarked
print(len(df_titanic))
df_titanic = df_titanic.dropna(subset=['Embarked'])
print(len(df_titanic))

In [ ]:
# Dumificamos variables categóricas
df_titanic_i = pd.get_dummies(df_titanic[cols], 
                              prefix_sep='_',
                              drop_first=True, 
                              columns=cat_cols)
df_titanic_i.head()

## 0105 Train-Test Split

In [ ]:
# Generamos las matrices X e y
X_titanic = df_titanic_i.drop('Survived',axis=1)
y_titanic = df_titanic_i['Survived']

X_titanic.shape, y_titanic.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic, 
                                                    y_titanic, 
                                                    test_size=0.25, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 02 Entrenamiento árbol de decisión
Entrenamos un árbol de decisión tomando medidas para evitar el overfitting.

In [ ]:
def entrenar_modelo_y_predecir_classificacion(modelo):
  # Entreno el árbol con el set de entrenamiento
  modelo = modelo.fit(X=X_train, y=y_train)
  # Uso el árbol para predecir sobre el dataset de entrenamiento
  y_pred_train = modelo.predict(X_train)
  # Uso el árbol para predecir sobre el dataset de test
  y_pred_test = modelo.predict(X_test)
  # Cómo de buena es la predicción?
  ac_train = round(accuracy_score(y_train, y_pred_train), 4)
  print('Precisión en set de entrenamiento :', ac_train)
  ac_test = round(accuracy_score(y_test, y_pred_test), 4)
  print('Precisión en set de test :', ac_test)
  print('Degradación: ', round((ac_train-ac_test)/ac_train*100,2), '%')

In [ ]:
modelo = tree.DecisionTreeClassifier(max_depth=20, 
                                     min_samples_split = 20,
                                     min_samples_leaf = 5, 
                                     min_impurity_decrease = 0.003)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

In [ ]:
# con export_graphviz
dot_data = tree.export_graphviz(modelo, 
                                out_file=None, 
                                feature_names=list(X_titanic.columns)) 
graph = graphviz.Source(dot_data) 
graph

# 03 Entrenamiento Random Forest
Ahora comparamos el rendimiento del árbol de decisión con un Random Forest.
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
# Importamos el modelo desde Sklearn
from sklearn import ensemble

In [ ]:
modelo = ensemble.RandomForestClassifier(n_estimators = 200,
                                         max_depth = 3, 
                                         min_samples_split = 10,
                                         min_samples_leaf = 5)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

In [ ]:
modelo = ensemble.RandomForestClassifier(n_estimators = 250,
                                         max_features = "auto",
                                         max_depth = 5, 
                                         min_samples_split = 5,
                                         min_samples_leaf = 3)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

# 04 EJERCICIO


## 0401 Importa Dataset Diabetes
Vamos a utilizar el dataset del notebook 3A (Diabetes).

Impórtalo utilizando el archivo raw: 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/diabetesIndia.csv'.

Guárdalo en el **dataframe df**.

In [ ]:
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/diabetesIndia.csv'

df = pd.read_csv(url)

df.head(2)

## 0402 Limpia el dataset
Comprueba el dataset (hay nulos) y prepáralo si es necesario.

In [ ]:
# Rellenamos los nulos de las variables numéricas (edad)
df.isna().sum()

## 0403 Train-Test split
Genera el set de pruebas (test).

In [ ]:
# Generamos las matrices X e y
X = df.drop('Outcome',axis=1)
y = df['Outcome']

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

## 0404 Árbol de Decisión
Encuentra el mejor árbol de decisión posible

In [ ]:
modelo = tree.DecisionTreeClassifier(max_depth=15, 
                                     min_samples_split = 20,
                                     min_samples_leaf = 10, 
                                     min_impurity_decrease = 0.005)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

## 0405 Random Forest
Encuentra el mejor modelo de rándom forest posible.

In [ ]:
modelo = ensemble.RandomForestClassifier(n_estimators = 500,
                                         max_features = "auto",
                                         max_depth = 3, 
                                         min_samples_split = 20,
                                         min_samples_leaf = 5, 
                                         min_impurity_decrease = 0.005)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)